In [10]:
import os
os.chdir('../rotaryWing/')
# This file was used to fill the Blade 2 Section of the table

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from tools.BET import BETheory
from tools.BEM import BEMTheory
from tools.airfoil import airfoil

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
roAir = 1.225   # kg/m3 density of air
roWater = 1000  # kg/m3
g = 9.8         # m/s2

# The airfoil : NACA 2412.
lift_slope = 2*np.pi

rBlade = 1.18 # m  Radius of the blade(rough estimate)
R_co = 0.2 # mine was 0.15
massOfStrutcutre= 10# kg this mass of everything other than payload mass.
omega_air = 700*2*np.pi/60 
omega_water = 28*2*np.pi/60
V = 0
# Given
massOfPayload = 5
volOfPayload = 0.0025
b = 2       # no of Blades . Tentatively decided on three
areaBlade = np.pi*(rBlade**2)

total_vol = 0.008 # The payload is gonna be taking a place inside this volume


In [13]:
def vel_power(V, medium='air'):
    if medium == 'air':
        T = (massOfStrutcutre + massOfPayload)*g; ro = roAir
    elif medium == 'water':
        ro = roWater
        T = (massOfStrutcutre + massOfPayload)*g - total_vol*ro*g
        
    ind_vel_MT = (-1*V/2) + np.sqrt(((V/2)**2) + (T/(2*ro*areaBlade)))
    power = T*(V + ind_vel_MT)
    return round(ind_vel_MT, 5) , round(power,4)
idealP_air = vel_power(V, 'air')[1]
ideaP_water = vel_power(V,'water')[1]
print(idealP_air, ideaP_water)

544.4224 6.0746


In [14]:
collective_pitch_air = 13*np.pi/180
collective_pitch_water = 16*np.pi/180
def linear_taper(r_ratio):
    return 0.1 - 0.03*r_ratio*rBlade
def linear_twist_air(r_ratio):
    return collective_pitch_air
def linear_twist_water(r_ratio):
    return collective_pitch_water
def drag_air(AoA):
    return 0.015
def drag_water(AoA):
    return 0.2

In [15]:
rotor_air = BEMTheory(
                angular_vel = omega_air, no_of_blades= b, 
                radius= rBlade, lift_slope=lift_slope, drag = drag_air, linear_twist=linear_twist_air, 
                climb_vel=V , root_cutouts=R_co, linear_taper=linear_taper)
rotor_water = BEMTheory(
                angular_vel = omega_water, no_of_blades= b, 
                radius= rBlade, lift_slope=lift_slope, drag = drag_water, linear_twist=linear_twist_water, 
                climb_vel=V , root_cutouts=R_co, linear_taper=linear_taper, medium='water')

In [16]:
print(f'Air : T,P = {rotor_air.T():.4f}, {rotor_air.P():.4f}')
print(f'Water : T,P = {rotor_water.T():.4f}, {rotor_water.P():.4f}')

Air : T,P = 341.4597, 665.8802
Water : T,P = 406.9592, 267.0623


In [17]:
for r_ratio in [0.18,0.3,0.4,0.45,0.5,0.6,0.68,0.75,0.8,0.9,0.95,0.98]:
    print(round(rotor_air.AoA(r_ratio)*180/np.pi,3), '-'*3,  round(rotor_water.AoA(r_ratio)*180/np.pi,3))

10.605 --- 6.655
11.142 --- 8.133
11.405 --- 8.972
11.504 --- 9.314
11.591 --- 9.619
11.734 --- 10.143
11.828 --- 10.505
11.899 --- 10.795
11.945 --- 10.997
12.027 --- 11.447
12.065 --- 11.796
12.092 --- 12.252
